# SimParam

> The SimParam class is the cornerstone of a simulation, acting as a global container for parameters that are not specific to any single Population but govern the rules of the entire simulation. Just as the Population class holds the state of individuals, SimParam holds the "genetic laws" of the simulated world, such as trait architecture, genetic maps, and SNP chip definitions.



In [ ]:
#| default_exp sp

In [ ]:
#| hide
from nbdev.showdoc import *

In [ ]:
#| export

import jax.numpy as jnp
from flax.struct import dataclass as flax_dataclass
from dataclasses import field
from typing import List, Optional, TYPE_CHECKING
# No need for cached_property anymore for these core attributes
# from functools import cached_property

@flax_dataclass(frozen=True)
class SimParam:
    """
    A container for all global simulation parameters.

    This object defines the static rules of the simulation, such as the
    genetic map and ploidy. It is created once at the beginning of a
    simulation and is treated as immutable.

    --- JAX JIT Compilation Notes ---
    ... (docstring remains the same) ...
    """
    # --- Core Genome Structure (Primary Inputs) ---
    gen_map: jnp.ndarray
    ploidy: int

    # --- Genetic Architecture ---
    traits: Optional['TraitCollection'] = None
    snp_chips: List['LociMap'] = field(default_factory=list)

    # --- Simulation Control ---
    sexes: str = "no"
    recomb_params: tuple = (2.6, 0.0, 0.0)
    n_threads: int = 1

    # --- Pedigree & History Tracking ---
    track_pedigree: bool = False
    track_recomb: bool = False
    pedigree: Optional[jnp.ndarray] = None
    last_id: int = 0 # Can be initialized to 0 now

    # --- Default Phenotyping Parameters ---
    var_e: Optional[jnp.ndarray] = None

    @classmethod
    def from_founder_pop(
        cls,
        founder_pop: 'Population',
        gen_map: jnp.ndarray,
        **kwargs
    ) -> 'SimParam':
        """
        Creates a SimParam object from a founder population and its genetic map.

        This is the recommended way to initialize simulation parameters. It
        ensures that the simulation's rules are perfectly aligned with the
        initial state of the first population.

        Args:
            founder_pop: The initial Population object.
            gen_map: A JAX array containing the genetic map.
            **kwargs: Additional SimParam attributes to set (e.g., sexes='yes').
        
        Returns:
            A new, fully configured SimParam object.
        """

        return cls(
            gen_map=gen_map,
            ploidy=founder_pop.geno.shape[2],
            last_id=founder_pop.nInd,
            **kwargs
        )

    # --- Derived Properties ---
    @property
    def centromere(self) -> jnp.ndarray:
        """Sets a default centromere position for each chromosome."""
        return jnp.zeros(self.n_chr)

    @property
    def n_chr(self) -> int:
        """Returns the number of chromosomes from the genetic map shape."""
        return self.gen_map.shape[0]

    @property
    def n_loci_per_chr(self) -> jnp.ndarray:
        """Returns an array with the number of loci for each chromosome."""
        return jnp.full((self.n_chr,), self.gen_map.shape[1])

    @property
    def n_traits(self) -> int:
        """Returns the number of defined traits."""
        if self.traits is None:
            return 0
        return self.traits.n_traits

    def __repr__(self) -> str:
        return (f"SimParam(nChr={self.n_chr}, nTraits={self.n_traits}, "
                f"ploidy={self.ploidy}, sexes='{self.sexes}')")

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()